<a href="https://colab.research.google.com/github/AndresMontesDeOca/Laboratorio3/blob/main/Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle Experiments

## Libraries

In [1]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook: # maquina virtual colab
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error


import warnings
# warnings.filterwarnings('ignore', category=ValueWarning)
warnings.filterwarnings('ignore')

# Asegurarte de que Pandas muestre los valores con la máxima precisión
pd.set_option('display.float_format', lambda x: '%.10f' % x)

# Ajustar la opción para mostrar más filas
# pd.set_option('display.max_rows', None)

# Si también quieres mostrar más columnas
# pd.set_option('display.max_columns', None)


# Vamos a suprimir la notacion cientifica
pd.set_option("display.float_format", lambda x:"%.2f" %x)


## Carga Datos

In [3]:
# Code to read csv file into Colaboratory:
# !pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

################################# Datasets ###################################
# # Ventas
id = "158aOjqxaNO8l97yA6VWJkek_15YVLMhs"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('sell-in.txt')
data_ventas = pd.read_csv("sell-in.txt", sep="\t")
data_ventas['periodo'] = pd.to_datetime(data_ventas['periodo'], format='%Y%m')
data_ventas['customer_id'] = data_ventas['customer_id'].astype(str)
data_ventas['product_id'] = data_ventas['product_id'].astype(str)
data = data_ventas.copy()

# # Productos
id = "15JS_k86LS0sgJXma7BOVXWlyNcMwxdhE"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_productos.txt')
data_productos = pd.read_csv("tb_productos.txt", sep="\t")
data_productos['product_id'] = data_productos['product_id'].astype(str)

# # Stocks
id = "15EV-8f_U7onpA1AcTxxXeD-z8yVR4fQu"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('tb_stocks.txt')
data_stocks = pd.read_csv("tb_stocks.txt", sep="\t")
data_stocks['periodo'] = pd.to_datetime(data_stocks['periodo'], format='%Y%m')
data_stocks['product_id'] = data_stocks['product_id'].astype(str)

# # Productos a predecir
id = "15LjADctFVwjzQFJvfJGFTEdgZx9xCoId"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('productos_a_predecir.txt')
data_productos_a_predecir = pd.read_csv("productos_a_predecir.txt", sep="\t")
data_productos_a_predecir['product_id'] = data_productos_a_predecir['product_id'].astype(str)



## Filter Data

In [4]:
def filter_data(data_all, data_filter):
    # Filtrar el DataFrame 'data_all' para que solo contenga los 'product_id' presentes en 'data_filter'
    data_filtered = data_all[data_all['product_id'].isin(data_filter['product_id'])]

    return data_filtered

## Group Data

In [5]:
# Siempre como values toma las toneladas vendidas
def group_data(data, column):
  grouped_data = data.groupby([column, 'periodo']).sum().reset_index()

  # Crea un DataFrame pivoteado donde las filas son las fechas y las columnas son los product_id
  pivot_data = grouped_data.pivot(index='periodo', columns=column, values='tn')

  # Asegúrate de que los nombres de las columnas sean strings
  pivot_data.columns = pivot_data.columns.astype(str)

  # Restablece el índice para asegurarse de que 'product_id' no sea un índice compuesto
  pivot_data.columns.name = None

  return pivot_data

## Fill Nulls

In [6]:
# Jugar con esto, no se si esta bien
def fill_nulls(df):
  # Primero usamos bfill para completar las ordenes mas viejas con los valores de las ordenes mas recientes
  df = df.bfill()
  # Luego completamos con ceros los productos que dejamos de vender, o se discontinuaron
  df = df.fillna(0)

  return df
###########################################################################
def fill_nulls_customer(df):
  df = df.fillna(0)
  return df


## Normalize Data

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def normalize_data(df, normalization="MinMax"):
    """
    Normaliza cada serie de tiempo (columna) de manera individual usando MinMax o Zscore.

    Args:
        df (pd.DataFrame): DataFrame con series de tiempo de distintos productos, cada columna es un producto.
        normalization (str): Tipo de normalización a aplicar. Opciones: "MinMax" o "Zscore". Default es "MinMax".

    Returns:
        normalized_df (pd.DataFrame): DataFrame con las series normalizadas.
        normalization_params (pd.DataFrame): DataFrame con los parámetros necesarios para desnormalizar cada columna.
            - Para "MinMax": valores min y max de cada columna.
            - Para "Zscore": valores mean y std de cada columna.
    """
    normalization_params = pd.DataFrame(columns=["product_id", "min", "max", "mean", "std"])
    normalized_df = pd.DataFrame(index=df.index)

    for column in df.columns:
        if normalization == "MinMax":
            scaler = MinMaxScaler()
            normalized_values = scaler.fit_transform(df[[column]]).flatten()
            new_params = pd.DataFrame({
                "product_id": [column],
                "min": [scaler.data_min_[0]],
                "max": [scaler.data_max_[0]],
                "mean": [None],
                "std": [None]
            })
            normalization_params = pd.concat([normalization_params, new_params], ignore_index=True)
            normalized_df[column] = normalized_values

        elif normalization == "ZScore":
            scaler = StandardScaler()
            normalized_values = scaler.fit_transform(df[[column]]).flatten()
            new_params = pd.DataFrame({
                "product_id": [column],
                "min": [None],
                "max": [None],
                "mean": [scaler.mean_[0]],
                "std": [scaler.scale_[0]]
            })
            normalization_params = pd.concat([normalization_params, new_params], ignore_index=True)
            normalized_df[column] = normalized_values

        else:
            raise ValueError("Invalid normalization method. Choose 'MinMax' or 'ZScore'.")

    return normalized_df, normalization_params

def denormalize_series(normalized_series, normalization_params, normalization="MinMax"):
    """
    Desnormaliza una serie de tiempo usando los valores almacenados.

    Args:
        normalized_series (pd.Series or pd.DataFrame): Serie o DataFrame con los datos normalizados.
        normalization_params (pd.DataFrame): DataFrame con los parámetros necesarios para desnormalizar cada serie o columna.
            - Para "MinMax": valores min y max de cada serie o columna.
            - Para "Zscore": valores mean y std de cada serie o columna.
        normalization (str): Tipo de normalización a deshacer. Opciones: "MinMax" o "Zscore". Default es "MinMax".

    Returns:
        denormalized_series (pd.Series or pd.DataFrame): Serie o DataFrame con los datos desnormalizados.
    """
    if isinstance(normalized_series, pd.DataFrame):
        denormalized_df = pd.DataFrame(index=normalized_series.index)
        for column in normalized_series.columns:
            params = normalization_params[normalization_params["product_id"] == column]
            if normalization == "MinMax":
                min_value = params["min"].values[0]
                max_value = params["max"].values[0]
                denormalized_values = normalized_series[column] * (max_value - min_value) + min_value
            elif normalization == "ZScore":
                mean_value = params["mean"].values[0]
                std_value = params["std"].values[0]
                denormalized_values = normalized_series[column] * std_value + mean_value
            else:
                raise ValueError("Invalid normalization method. Choose 'MinMax' or 'ZScore'.")
            denormalized_df[column] = denormalized_values
        return denormalized_df
    elif isinstance(normalized_series, pd.Series):
        product_ids = normalized_series.index
        denormalized_values = []
        for product_id in product_ids:
            params = normalization_params[normalization_params["product_id"] == product_id]
            if normalization == "MinMax":
                min_value = params["min"].values[0]
                max_value = params["max"].values[0]
                denormalized_value = normalized_series[product_id] * (max_value - min_value) + min_value
            elif normalization == "ZScore":
                mean_value = params["mean"].values[0]
                std_value = params["std"].values[0]
                denormalized_value = normalized_series[product_id] * std_value + mean_value
            else:
                raise ValueError("Invalid normalization method. Choose 'MinMax' or 'ZScore'.")
            denormalized_values.append(denormalized_value)
        denormalized_series = pd.Series(denormalized_values, index=product_ids, name=normalized_series.name)
        return denormalized_series
    else:
        raise TypeError("normalized_series should be either a pandas Series or DataFrame")


## Split Data

In [27]:
###############################################################################
# Hay que ponerse a invstigar sobre como hacer el Split en Series de Tiempo. Walk Fwd Validation?
def split_data_1(df):
  df_train = df.loc['2017-01':'2018-12']
  df_valid = df.loc['2019-01':'2019-12']
  return df_train, df_valid
###############################################################################
# No me deja un data_valid mas chico que esto, tira erro el earlystopping
def split_data_2(df):
  df_train = df.loc['2017-01':'2019-04']
  df_valid = df.loc['2019-05':]
  return df_train, df_valid
#############################################################################
def split_data_dec2019(df):
  df_train = df.loc['2017-01':'2019-10']
  df_valid = df.loc['2019-11':'2019-12']
  return df_train, df_valid

## Window Data

In [9]:
def window_dataset(sequence, data_split, window_size, batch_size, n_future, shuffle_buffer=1000, seed=None):
    """Generates dataset windows for multi-step forecasting in a multivariable context.

    Args:
      sequence (array-like): Contains the values of the time series, where each element is an array of feature values.
      data_split (str): Specifies if the dataset is for training or validation/test.
      window_size (int): The number of time steps to include in the feature.
      batch_size (int): The batch size.
      n_future (int): The number of future steps to predict.
      shuffle_buffer (int): Buffer size to use for the shuffle method.
      seed (int, optional): Random seed for reproducibility.

    Returns:
      tf.data.Dataset: TF Dataset containing time windows.
    """

    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(sequence)

    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + n_future, shift=1, drop_remainder=True)

    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + n_future))

    # Create tuples with features and labels
    dataset = dataset.map(lambda window: (window[:window_size], window[window_size:]))

    if data_split == 'train':
        # Shuffle the training data to improve generalization
        dataset = dataset.shuffle(shuffle_buffer, seed=seed)
    else:
        # Cache the validation/test data for improved performance
        dataset = dataset.cache()

    # Create batches of windows and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset


## Prediction Functions

In [47]:
def generate_predictions(data_norm, data_norm_params):

    # Convertir el DataFrame a un array de NumPy
    data_norm_array = data_norm.values

    # Extraer la última ventana de datos de 2019 para predecir enero de 2020
    column_names = data_norm.columns  # Obtener los nombres de las columnas

    # Extraer los últimos `window_size` meses de 2019
    input_data = data_norm_array[-window_size:].reshape((1, window_size, n_features))

    # Predecir enero de 2020
    pred_january = model.predict(input_data)

    # Asegurarse de que la predicción tenga la forma correcta
    pred_january = pred_january.reshape((1, n_future, n_features))

    # Crear un DataFrame para la predicción de enero de 2020
    pred_january_df = pd.DataFrame(pred_january[0], columns=column_names)
    pred_january_df.index = pd.date_range(start='2020-01-01', periods=n_future, freq='MS')

    # Actualizar la ventana de entrada para predecir febrero de 2020
    input_data = np.append(input_data[:, 1:, :], pred_january[:, 0, :].reshape(1, 1, n_features), axis=1)

    # Predecir febrero de 2020
    pred_february = model.predict(input_data)

    # Asegurarse de que la predicción tenga la forma correcta
    pred_february = pred_february.reshape((1, n_future, n_features))

    # Crear un DataFrame para la predicción de febrero de 2020
    pred_february_df = pd.DataFrame(pred_february[0], columns=column_names)
    pred_february_df.index = pd.date_range(start='2020-02-01', periods=n_future, freq='MS')

    # Obtener la predicción de febrero de 2020
    pred_1 = pred_january_df.loc['2020-02-01']

    # Desnormalizar la predicción
    pred_1_denorm = denormalize_series(pred_1, data_norm_params, normalization=normalization)
    data_pred1_denorm = pred_1_denorm.reset_index()
    data_pred1_denorm.columns = ['product_id', 'tn']
    predicciones = filter_data(data_pred1_denorm, data_productos_a_predecir)

    # Crear el nombre del archivo
    filename = f"{split_strategy}_{model_name}_win{window_size}_batch{batch_size}_{normalization}_{loss}_epochs{epochs}.csv"
    predicciones.to_csv(filename, header=True, index=False)

    print(filename)

    return predicciones
##########################################################################
def sumar_predicciones(df1, df2):
    # Asegúrate de que las columnas necesarias estén en los DataFrames
    if 'product_id' not in df1.columns or 'tn' not in df1.columns:
        raise ValueError("df1 debe contener las columnas 'product_id' y 'tn'")
    if 'product_id' not in df2.columns or 'tn' not in df2.columns:
        raise ValueError("df2 debe contener las columnas 'product_id' y 'tn'")

    # Suma los valores de 'tn' para cada 'product_id' de ambos DataFrames
    result = df1.set_index('product_id').add(df2.set_index('product_id'), fill_value=0).reset_index()

    return result
##########################################################################

## Complete Sales

In [11]:
def complete_sales(df1, df2):
    # Crear un rango de fechas desde enero 2017 hasta diciembre 2019
    fechas_completas = pd.date_range(start='2017-01-01', end='2019-12-01', freq='MS')

    # Reindexar el DataFrame para asegurar que todas las fechas estén presentes
    df1 = df1.reindex(fechas_completas, fill_value=pd.NA)

    # Obtener los product_id del primer DataFrame (nombres de las columnas)
    product_ids_df1 = df1.columns.tolist()

    # Obtener los product_id del segundo DataFrame (valores en la columna 'product_id')
    product_ids_df2 = df2['product_id'].tolist()

    # Identificar los product_id que faltan en df1
    product_ids_faltantes = [pid for pid in product_ids_df2 if pid not in product_ids_df1]

    # Crear un DataFrame con las columnas faltantes y valores NaN
    df_faltantes = pd.DataFrame(index=df1.index, columns=product_ids_faltantes)

    # Concatenar el DataFrame original con el DataFrame de faltantes
    df_resultante = pd.concat([df1, df_faltantes], axis=1)

    return df_resultante



## Callbacks

In [12]:
# Por ahora solo EarlyStopping
#############################################################################
class MAEThresholdCallback(Callback):
    def __init__(self, threshold=0.15):
        super(MAEThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None):
        val_mae = logs.get('val_mae')
        if val_mae is not None and val_mae <= self.threshold:
            print(f'\nEpoch {epoch+1}: Validation MAE has reached {val_mae:.4f}, stopping training.')
            self.model.stop_training = True

def MyCallbacks(model_name, patience):
    earlystop = tf.keras.callbacks.EarlyStopping('val_loss', patience=patience, restore_best_weights=True)
    # checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=f'ckpts/{model_name}-' + '{epoch:02d}-{val_loss:.4f}.h5', monitor='val_loss')
    # mae_threshold_callback = MAEThresholdCallback(threshold=0.015)
    return [earlystop] #, checkpoint] #, mae_threshold_callback]

#############################################################################

## Model Design

In [13]:
#############################################################################
def compile_model(new_model, loss, optimizer):
  new_model.compile(optimizer=optimizer, loss=loss) # metrics=[tf.keras.metrics.MeanAbsolutePercentageError()])
  print(new_model.summary())
  return new_model
#############################################################################
def MyModel(loss, optimizer, window_size, n_future, n_features):
    new_model = tf.keras.Sequential([
        tf.keras.layers.InputLayer((window_size, n_features)),
        tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='causal'),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=False)),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(n_features * n_future, activation='relu'),
        tf.keras.layers.Reshape((n_future, n_features)),
        ])
    return compile_model(new_model, loss, optimizer)

# Pipelines

## Data Preprocessing

In [116]:
# # data
# # data_productos
# # data_stocks
# # data_productos_a_predecir

# Pre-Processing Variables
split_strategy = 'S3'
window_size = 6
n_future = 2
n_features = len(data_productos_a_predecir)
batch_size = 4
normalization = 'MinMax'

data_filter = filter_data(data, data_productos_a_predecir)
data_grouped = group_data(data_filter, 'product_id')
data_filled = fill_nulls(data_grouped)
data_norm, data_norm_params = normalize_data(data_filled, normalization=normalization)

## Experimentos

### 06/11 - Analisis del Error por Producto

In [ ]:
# # Pre-Processing Variables
# split_strategy = 'S3'
# window_size = 6
# n_future = 2
# n_features = len(data_productos_a_predecir)
# batch_size = 4
# normalization = 'MinMax'

# data_filter = filter_data(data, data_productos_a_predecir)
# data_grouped = group_data(data_filter, 'product_id')
# data_filled = fill_nulls(data_grouped)
# data_norm, data_norm_params = normalize_data(data_filled, normalization=normalization)


# # # # #########################################################################
# # # # Train hasta 2019-10, para predecir 2019-12
# # # # #########################################################################
data_train, data_valid = split_data_dec2019(data_norm)
data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)

# Model Variables
model_name = 'Clase3'
loss = 'mse'
optimizer = 'adam'
patience = 30
epochs = 10

callbacks = MyCallbacks(model_name, patience)
model = MyModel(loss, optimizer, window_size, n_future, n_features)

history = model.fit(
    data_train_windowed,
    # validation_data = data_valid_windowed,
    # callbacks = callbacks,
    verbose=2,
    epochs=epochs)

# No sirve para este caso, esta hardcodeada a Febrero 2020. Tengo que actualizar esta funcion
# generate_predictions(data_norm, data_norm_params)

# Seleccionar los últimos 6 meses de data_train
data_for_prediction = data_train[-window_size:]
# Convierte los datos a un formato compatible con la función window_dataset
data_for_prediction = data_for_prediction.values.reshape((1, window_size, n_features))
predictions = model.predict(data_for_prediction)

# Convertir las predicciones a un DataFrame para desnormalizar
predictions_df = pd.DataFrame(predictions[0], columns=data_train.columns)

# Desnormalizar las predicciones
predictions_denorm = denormalize_series(predictions_df, data_norm_params, normalization=normalization)

# Imprimir las predicciones desnormalizadas
# display(predictions_denorm)

# Exporto a formato Kaggle, para probar un tirito (no sirve porque Kaggle valida a Febrero 2020)
predictions_denorm.iloc[1].to_csv('predictions_dec2019.csv', header=True, index=True)

# Genero las Series para plotear el error entre predicho y real
data_dec2019_pred = pd.Series(predictions_denorm.iloc[1], name='Pred')
data_dec2019_true = pd.Series(data_grouped.fillna(0).loc['2019-12-01'], name='True')

# Ploteo
sns.scatterplot(x=data_dec2019_pred, y=data_dec2019_true)
plt.axline((0, 0), slope=1, color='r', linestyle='--')
plt.show()

# Productos con mayores diferencias en la prediccion
print('Productos con peores predicciones:\n', abs(data_dec2019_pred - data_dec2019_true).sort_values(ascending=False).head(10))


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_9 (Conv1D)           (None, 6, 64)             149824    
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 3, 64)             0         
 g1D)                                                            
                                                                 
 bidirectional_18 (Bidirect  (None, 3, 64)             24832     
 ional)                                                          
                                                                 
 bidirectional_19 (Bidirect  (None, 32)                10368     
 ional)                                                          
                                                                 
 dropout_9 (Dropout)         (None, 32)                0         
                                                      

### 06/10 - Cada cliente por separado

In [15]:
# Esto lo corro en mi maquina local, en Colab se cuelga antes de terminar con los casi 600 clientes

# # Inicializo el vector de predicciones
# predicciones_all = data_productos_a_predecir.copy()
# predicciones_all['tn'] = 0

# # Probamos solo con los 3 primeros clientes
# customers = data['customer_id'].unique()
# i = 0

# for customer in customers:
#   print('Vuelta: ', i)
#   i += 1
#   data_customer = data_filter.query('customer_id == @customer')
#   data_customer_grouped = group_data(data_customer, 'product_id')
#   data_customer_grouped_fixed = complete_sales(data_customer_grouped, data_productos_a_predecir)
#   data_customer_filled = fill_nulls(data_customer_grouped_fixed) # Probar cual funciona mejor
#   # data_customer_filled = fill_nulls_customer(data_customer_grouped_fixed) # Probar cual funciona mejor
#   data_customer_norm, data_customer_norm_params = normalize_data(data_customer_filled, normalization=normalization)

#   data_train_windowed = window_dataset(data_customer_norm, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)

#   # Model Variables
#   model_name = 'M1'
#   loss = 'mse'
#   optimizer = 'adam'
#   patience = 30
#   epochs = 10

#   model = MyModel(loss, optimizer, window_size, n_future, n_features)

#   history = model.fit(
#       data_train_windowed,
#       # validation_data = data_valid_windowed,
#       # callbacks = callbacks,
#       verbose=2,
#       epochs=epochs)

#   predicciones = generate_predictions(data_customer_norm, data_customer_norm_params)

#   predicciones_all = sumar_predicciones(predicciones_all, predicciones)

# predicciones_all.to_csv('/content/drive/MyDrive/Universidad Austral (Resumenes)/predicciones2.csv', header=True, index=False)

### 06/10 - No Split

In [16]:
# # # #########################################################################
# # # Sin Splitear los datos, usando todo para entrenar
# # # #########################################################################

#  data_train_windowed = window_dataset(data_norm, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# # data_train_windowed = window_dataset(data_norm, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)

# # Model Variables
# model_name = 'M1'
# loss = 'mse'
# optimizer = 'adam'
# patience = 30
# epochs = 10

# model = MyModel(loss, optimizer, window_size, n_future, n_features)


# history = model.fit(
#     data_train_windowed,
#     # validation_data = data_valid_windowed,
#     # callbacks = callbacks,
#     verbose=2,
#     epochs=epochs)

# predicciones = generate_predictions()

### 06/10 - Split #2

In [17]:
# # # #########################################################################
# # # Train desde 2017-01 hasta 2019-06
# # # #########################################################################

# data_train, data_valid = split_data_2(data_norm)
# data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)

# # Model Variables
# model_name = 'M1'
# loss = 'mse'
# optimizer = 'adam'
# patience = 30
# epochs = 500

# callbacks = MyCallbacks(model_name, patience)
# model = MyModel(loss, optimizer, window_size, n_future, n_features)


# history = model.fit(
#     data_train_windowed,
#     validation_data = data_valid_windowed,
#     callbacks = callbacks,
#     verbose=2,
#     epochs=epochs)

# generate_predictions()

### 06/10 - Split #1

In [18]:
# # # #########################################################################
# # # Train 2018 & 2018, Validation 2019 (10/06)
# # # #########################################################################
# data_train, data_valid = split_data_1(data_norm)
# data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)

# # Model Variables
# model_name = 'M1'
# loss = 'mse'
# optimizer = 'adam'
# patience = 30
# epochs = 500

# callbacks = MyCallbacks(model_name, patience)
# model = MyModel(loss, optimizer, window_size, n_future, n_features)

# history = model.fit(
#     data_train_windowed,
#     validation_data = data_valid_windowed,
#     callbacks = callbacks,
#     verbose=2,
#     epochs=epochs)

# generate_predictions()

### Univariate: Product 20001

In [19]:
# data = data_filled[['20001']]

# # Parámetros
# window_size = 6  # Tamaño de la ventana de tiempo
# n_future = 2  # Número de pasos futuros a predecir
# batch_size = 32
# n_splits = 5  # Número de divisiones para validación "walk forward"
# n_features = data.shape[1]  # Número de características en el conjunto de datos

# data_norm, data_norm_params = normalize_data(data, normalization=normalization)
# data_norm

### Proof of Concept

In [20]:
# #########################################################################
# # New Pipeline (09/06)
# #########################################################################
# data_norm, data_norm_params = normalize_data(data_filled, normalization=normalization)
# # data_train, data_valid = split_data(data_norm) # Split pendiente
# data_train = data_norm
# print(data_train.shape)
# # print(data_valid.shape)
# data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# # data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)


# #########################################################################
# # Old Pipeline (08/06)
# #########################################################################
# # data_all = group_data(data, data_productos_a_predecir)
# # data_all_norm, data_all_norm_params = normalize_data(data_all, normalization=normalization)
# # data_all_norm['20001'].describe()
# # data_train, data_valid = split_data_all(data_all_norm)
# # print(data_train.shape)
# # print(data_valid.shape)
# # data_train = data_all_norm
# # data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# # data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)


# #########################################################################
# # Modelo
# #########################################################################
# data_train, data_valid = split_data_1(data_norm)
# data_train_windowed = window_dataset(data_train, data_split='train', window_size=window_size, batch_size=batch_size, n_future=n_future)
# data_valid_windowed = window_dataset(data_valid, data_split='valid', window_size=window_size, batch_size=batch_size, n_future=n_future)

# # Model Variables
# model_name = 'M1'
# loss = 'mse'
# optimizer = 'adam'
# patience = 30
# epochs = 500

# callbacks = MyCallbacks(model_name, patience)
# model = MyModel(loss, optimizer, window_size, n_future, n_features)

# history = model.fit(
#     data_train_windowed,
#     validation_data = data_valid_windowed,
#     callbacks = callbacks,
#     verbose=2,
#     epochs=epochs)

# generate_predictions()

# EDA

In [21]:
# # Evidentemente cuando el profe deidentifico los customers, lo hizo asignandoles ID secuenciales al listado ordenado por la suma de ventas(tn)
# print('Listado de Clientes, ordenados por la sumatoria de ventas en tn:\n', group_data(data, 'customer_id').sum(), '\n')

# # Lo mismo cuando deidentifico a los productos, solo que esta vez empezo desde 20000
# print('Listado de Productos, ordenados por la sumatoria de ventas en tn:\n', group_data(data, 'product_id').sum())

In [22]:
# # Analisis del cliente top

# data_top = data.query('customer_id == "10001"')
# data_top = group_data(data_top, 'product_id')
# data_top = fill_nulls(data_top)

# # data_top